In [1]:
import pandas as pd

In [2]:
def load_data(path="../data/csv/all.csv"):
    return pd.read_csv(path, index_col=0)

In [3]:
df = load_data()

In [4]:
df.head()

,jobId,read_sum,write_sum,readSyscalls_sum,writeSyscalls_sum,readReal_sum,writeReal_sum,writeCancelled_sum,rxBytes_sum,rxPackets_sum,...,mem.used,mem.active,mem.available,mem.buffers,mem.cached,mem.slab,mem.buffcache,mem.swaptotal,mem.swapused,mem.swapfree
0,-8Ip2oamm-1-10,1.640063e+07,1865.0,20775.0,37.0,3.399680e+05,69632.0,0.0,0.0,0.0,...,3326259200,941318144,7053393920,2768896,2302001152,590626816,2895396864,0,0,0
1,-8Ip2oamm-1-11,3.943547e+09,99029.0,863138.0,1382.0,2.988442e+09,4825088.0,0.0,506728.0,1192.0,...,3656019968,1019281408,7034150912,2768896,2545008640,594448384,3142225920,0,0,0
2,-8Ip2oamm-1-12,2.260023e+09,63886.0,561366.0,906.0,1.886495e+09,2158592.0,0.0,341954.0,803.0,...,3795075072,995864576,7057567744,2768896,2772025344,480731136,3255525376,0,0,0
3,-8Ip2oamm-1-13,2.284814e+09,147790.0,582911.0,29318.0,4.472832e+06,3805184.0,0.0,354552.0,832.0,...,4242853888,1047011328,6947700736,2768896,3209531392,467460096,3679760384,0,0,0
4,-8Ip2oamm-1-14,2.754452e+09,70983.0,628488.0,986.0,2.200519e+09,3858432.0,0.0,380100.0,894.0,...,3600007168,966131712,7087300608,2768896,2544816128,590618624,3138203648,0,0,0


In [5]:
df.columns

Index(['jobId', 'read_sum', 'write_sum', 'readSyscalls_sum',
       'writeSyscalls_sum', 'readReal_sum', 'writeReal_sum',
       'writeCancelled_sum', 'rxBytes_sum', 'rxPackets_sum', 'rxErrors_sum',
       'rxDrop_sum', 'rxFifo_sum', 'rxFrame_sum', 'rxCompressed_sum',
       'rxMulticast_sum', 'txBytes_sum', 'txPackets_sum', 'txErrors_sum',
       'txDrop_sum', 'txFifo_sum', 'txColls_sum', 'txCarrier_sum',
       'txCompressed_sum', 'cpu_mean', 'cpu_max', 'memory_mean', 'memory_max',
       'ctime_mean', 'ctime_max', 'ctime_sum', 'workflowName', 'size',
       'executable', 'args', 'inputs', 'outputs', 'name', 'command',
       'execTimeMs', 'cpu.manufacturer', 'cpu.brand', 'cpu.vendor',
       'cpu.family', 'cpu.model', 'cpu.stepping', 'cpu.speed', 'cpu.cores',
       'cpu.physicalCores', 'cpu.processors', 'cpu.cache.l1d', 'cpu.cache.l1i',
       'cpu.cache.l2', 'cpu.cache.l3', 'mem.total', 'mem.free', 'mem.used',
       'mem.active', 'mem.available', 'mem.buffers', 'mem.cached', 'mem

In [6]:
def normalize_column(column):
    minvalue, maxvalue = column.min(), column.max()
    return (column - minvalue) / (maxvalue - minvalue)

def vectorize_textual(column):
    return pd.get_dummies(column)

df.select_dtypes(include='object')
df.ctime_max.mean(), df.execTimeMs.mean()

(11051.542998600797, 12287.035978192425)

## I co dalej
Było
- workflow
- size
- inputs
- outputs
- name
- cpu manufacturer
- cpu brand
- cpu speed
- cpu cores
- cpu physical cores
- mem_total
- mem_active

Wynikowo:
- memory_max <- czy memory intensive
- execTimeMs <- czy cpu intensive
- read_sum <- czy read intensive
- write_sum <- czy write intensive

Czyli - wektoryzujemy wszystko oprócz memory_max, execTimeMs, read_sum i write_sum

In [7]:
df.select_dtypes(exclude='object').dtypes

read_sum              float64
write_sum             float64
readSyscalls_sum      float64
writeSyscalls_sum     float64
readReal_sum          float64
writeReal_sum         float64
writeCancelled_sum    float64
rxBytes_sum           float64
rxPackets_sum         float64
rxErrors_sum          float64
rxDrop_sum            float64
rxFifo_sum            float64
rxFrame_sum           float64
rxCompressed_sum      float64
rxMulticast_sum       float64
txBytes_sum           float64
txPackets_sum         float64
txErrors_sum          float64
txDrop_sum            float64
txFifo_sum            float64
txColls_sum           float64
txCarrier_sum         float64
txCompressed_sum      float64
cpu_mean              float64
cpu_max               float64
memory_mean           float64
memory_max            float64
ctime_mean            float64
ctime_max             float64
ctime_sum             float64
size                  float64
execTimeMs            float64
cpu.family            float64
cpu.model 

In [55]:
def results_frame(dataframe):
    return dataframe.filter(['execTimeMs', 'cpu_mean', 'read_sum', 'write_sum', 'memory_max'])

def features_frame(dataframe):
    filtered = dataframe.filter(['workflowName', 'size', 'executable', 'args', 'inputs', 'outputs', 'name', 'cpu.manufacturer', 'cpu.brand', 'cpu.vendor',
                               'cpu.family', 'cpu.model', 'cpu.stepping', 'cpu.speed', 'cpu.cores', 'cpu.physicalCores', 'cpu.processors', 'cpu.cache.l1d', 
                               'cpu.cache.l1i', 'cpu.cache.l2', 'cpu.cache.l3', 'mem.total', 'mem.free', 'mem.used', 'mem.active', 'mem.available', 'mem.buffers', 
                               'mem.cached', 'mem.slab', 'mem.buffcache', 'mem.swaptotal', 'mem.swapused', 'mem.swapfree'])
    dataframe = drop_null_columns(filtered)
    dataframe = drop_boring_columns(dataframe)    
    
    vectorize_list_columns(dataframe, ['args', 'inputs', 'outputs'])
    
    numerical_columns = dataframe.select_dtypes(include='number').columns  
    vectorize_numerical_columns(dataframe, numerical_columns)
    
    textual_columns = dataframe.select_dtypes(include="object").columns
    dataframe = vectorize_textual_columns(dataframe, textual_columns)
    return dataframe
    
def drop_null_columns(dataframe):
    null_columns = get_columns_with_nulls(dataframe)
    return dataframe.drop(null_columns, axis=1)

def drop_boring_columns(dataframe):
    boring_columns = get_non_variable_columns(dataframe)
    return dataframe.drop(boring_columns, axis=1)
    
def vectorize_list_columns(dataframe, columns):
    for column in columns:
        dataframe[column] = vectorize_list(dataframe[column])
        
def vectorize_numerical_columns(dataframe, columns):
    for column in columns:
        dataframe[column] = normalize_column(dataframe[column])
        
def vectorize_textual_columns(dataframe, columns):
    encoded = [pd.get_dummies(dataframe[column]) for column in columns]    
    return pd.concat([dataframe.drop(columns, axis=1), *encoded], axis=1)
    
def vectorize_list(series):
    def vectorize(list_string):
        return len(eval(list_string))
    return series.apply(vectorize)
    
def get_columns_with_nulls(dataframe):
    columns = list(dataframe.columns)
    return [column for column in columns if dataframe[column].isnull().values.any()]

def get_non_variable_columns(dataframe):
    columns = list(dataframe.columns)
    return [column for column in columns if dataframe[column].unique().size == 1]

In [73]:
test_df = df
features_df = features_frame(test_df)
results_df = results_frame(test_df)

In [74]:
features_df.columns

Index(['size', 'args', 'inputs', 'outputs', 'cpu.speed', 'cpu.cores',
       'cpu.physicalCores', 'mem.total', 'mem.free', 'mem.used', 'mem.active',
       'mem.available', 'mem.buffers', 'mem.cached', 'mem.slab',
       'mem.buffcache', 'montage', 'montage2', 'soykb', 'bwa-wrapper',
       'gatk-wrapper', 'mAdd', 'mBackground', 'mBgModel', 'mConcatFit',
       'mDiffFit', 'mImgtbl', 'mJPEG', 'mProject', 'mProjectPP', 'mShrink',
       'mViewer', 'picard-wrapper', 'samtools-wrapper', 'add_replace',
       'alignment_to_reference', 'bwa-index', 'combine_variants', 'dedup',
       'faidx', 'filtering_indel', 'filtering_snp', 'genotype_gvcfs',
       'haplotype_caller', 'indel_realign', 'mAdd', 'mBackground', 'mBgModel',
       'mConcatFit', 'mDiffFit', 'mImgtbl', 'mJPEG', 'mProject', 'mProjectPP',
       'mShrink', 'mViewer', 'merge_gcvf', 'realign_target_creator',
       'select_variants_indel', 'select_variants_snp', 'seq_dict', 'sort_sam',
       'AMD', 'Intel®', 'EPYC 7B12', 'Xeon®',

In [75]:
from scipy.stats import percentileofscore

In [76]:
def calculate_quantile_rank(labels, label):
    return percentileofscore(labels, label) / 100

def calculate_utilization_class(labels, label):
    def label_for_rank(rank):
        if rank > 0.75:
            return 'very high'
        elif rank > 0.5:
            return 'high'
        elif rank > 0.25:
            return 'medium'
        else:
            return 'low'
    return label_for_rank(calculate_quantile_rank(labels, label))

In [116]:
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

In [78]:
cpu_means = results_df['cpu_mean']

X = features_df
y = cpu_means.map(lambda value: calculate_utilization_class(cpu_means, value))

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1) # 70% training and 30% test

In [81]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9761704875686148


In [83]:
def perform_classification_trial(classifier, X_train, y_train, X_test, y_test, classifier_name):
    print(f"Evaluating classifier '{classifier_name}'")
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(f"{classifier_name} has accuracy of {metrics.accuracy_score(y_test, y_pred)}")
    return y_pred

In [84]:
perform_classification_trial(DecisionTreeClassifier(), X_train, y_train, X_test, y_test, "Decision tree classifier")

Evaluating classifier 'Decision tree classifier'
Decision tree classifier has accuracy of 0.9767301689807341


array(['very high', 'medium', 'medium', ..., 'medium', 'medium', 'medium'],
      dtype=object)

In [86]:
cpu_means = results_df['cpu_mean']

X = features_df
y = cpu_means.map(lambda value: calculate_quantile_rank(cpu_means, value))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1) # 70% training and 30% test

In [106]:
def perform_regression_trial(regressor, X_train, y_train, X_test, y_test, regressor_name):
    print(f"Evaluating regressor '{regressor_name}'")
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    print(f"{regressor_name} accuracy:\n\tMAE: {metrics.mean_absolute_error(y_test, y_pred)}")

In [118]:
perform_regression_trial(DecisionTreeRegressor(random_state = 0), X_train, y_train, X_test, y_test, 'Decision tree regressor')
perform_regression_trial(KNeighborsRegressor(), X_train, y_train, X_test, y_test, 'KNN regressor')
perform_regression_trial(RandomForestRegressor(random_state = 0, n_estimators=20), X_train, y_train, X_test, y_test, 'Random forest regressor')
perform_regression_trial(MLPRegressor(random_state = 0), X_train, y_train, X_test, y_test, 'NN regressor')
perform_regression_trial(GradientBoostingRegressor(random_state = 0), X_train, y_train, X_test, y_test, 'Gradient Boosting regressor')

Evaluating regressor 'Decision tree regressor'
Decision tree regressor accuracy:
	MAE: 0.01176865119032326
Evaluating regressor 'KNN regressor'
KNN regressor accuracy:
	MAE: 0.012356870936388088
Evaluating regressor 'Random forest regressor'
Random forest regressor accuracy:
	MAE: 0.012209567227139322
Evaluating regressor 'NN regressor'
NN regressor accuracy:
	MAE: 0.03152996664527071
Evaluating regressor 'Gradient Boosting regressor'
Gradient Boosting regressor accuracy:
	MAE: 0.03666032646748087
